In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.stat import Correlation
#import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
#import holidays
from datetime import datetime, timezone
from pyspark.ml.classification import RandomForestClassifier, BinaryLogisticRegressionSummary
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.pipeline import PipelineModel

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("US_Accidents") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR") #supress warnings

24/11/18 16:43:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/18 16:43:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Read in Data
df = spark.read.parquet("final_dataset_revised.parquet")

In [4]:
# Combine Columns Percent_Age_15-19 and Percent_Age_20-24
df = df.withColumn(
    "Percent_Age_15-24",
    col("Percent_Age_15-19") + col("Percent_Age_20-24")
)

df = df.drop('Percent_Age_15-19')
df = df.drop('Percent_Age_20-24')

In [5]:
# Add Severity Binary Column
df = df.withColumn('Severity_Binary', when((col("Severity")==1) | (col("Severity")==2), 0).otherwise(1))

In [6]:
# Save to Parquet File
df.write.parquet("updated_dataset.parquet")